# Web scraping: Tableau comparatif
###  http://johnhwman.free.fr/Divers/Comparatif_VE.pdf

## Imports

In [6]:
import pandas as pd
import tabula as tb

## Code

In [7]:
# Nous décomposons les tables du pdf.
file = 'comparatif_VE.pdf'
data1 = tb.read_pdf(file, area = (32, 0,1000, 100000), pages = '1')
data2 = tb.read_pdf(file, area = (0, 0,700, 100000), pages = '2')
data3 = tb.read_pdf(file, area = (0, 0,100, 100000), pages = '3')

In [8]:
# Nous renomons les colonnes
column_name = ['Marque / Modèle', 'Capacité Batterie (brute / utile)', 'Puissance/couple moteur(s) ch / kW / Nm', 'Temps 0-100km/h', 'Vitesse maximale', 'Poids à Vide/Cx/Scx', 'Autonomie/consommation constructeur (cycle WLTP)', 'Estimation autonomie / conso. réelle en ville (à 23°C et -10°C)', 'Estimation autonomie / conso. réelle autoroute (à 23°C et -10°)', 'Modes de charge / puissances', 'Coût recharge 100% de la batterie', 'Coût au 100km (WLTP)', 'Prix du véhicule (TTC - Bonus écologique 6k€ ou 2k€ Inclus)']
data3[0].columns= column_name
data1[0].columns= column_name
data2[0].columns= column_name

In [9]:
# On concatène les 3 tables et on enlève les valeurs nulles
liste_concat = [data1[0],data2[0],data3[0]]
df_pdf = pd.concat(liste_concat).dropna()
df_pdf = df_pdf.reset_index()
df_pdf = df_pdf.drop('index', axis=1)

In [10]:
#Utilisation du Regex pour séparer les colonnes contenant plusieurs valeurs
df_pdf[['Capacité Batterie brute', 'Capacité Batterie utile']] = df_pdf['Capacité Batterie (brute / utile)'].str.split('/', expand=True)
df_pdf[['Puissance/couple moteur(s) ch', 'Puissance/couple moteur(s) kW','Puissance/couple moteur(s) Nm']] = df_pdf['Puissance/couple moteur(s) ch / kW / Nm'].str.split('/', expand=True)
df_pdf[['Poids à Vide', 'Poids Cx','Poids Scx']] = df_pdf['Poids à Vide/Cx/Scx'].str.split('/', expand=True)
df_pdf[['Autonomie', 'Consommation construsteur (cycle WLTP)']] = df_pdf['Autonomie/consommation constructeur (cycle WLTP)'].str.extract(r'([\d]+ km) / ([\d,]+ Wh/km)')
df_pdf[['Estimation autonomie ville','conso. réelle en ville (à 23°C et -10°C)']]=df_pdf['Estimation autonomie / conso. réelle en ville (à 23°C et -10°C)'].str.extract(r'(\d+-\d+ km) / (\d+-\d+ Wh/km)')
df_pdf[['Estimation autonomie autoroute','conso. réelle autoroute (à 23°C et -10°)']]= df_pdf['Estimation autonomie / conso. réelle autoroute (à 23°C et -10°)'].str.extract(r'(\d+-\d+ km) / (\d+-\d+ Wh/km)')
df_pdf['Autonomie'] = df_pdf['Autonomie'].str.replace(r'[^\d.]', '', regex=True)

In [11]:
# On transforme et on extraie les valeurs numériques de la colonne Côut au 100km 
def transform_and_extract_numeric(value):
    first_part = value.split('/')[0]
    numeric_part = ''.join(char for char in first_part if char.isdigit() or char == ',')
    numeric_part = numeric_part.replace(',', '.')
    return float(numeric_part) if numeric_part else None

In [12]:
df_pdf['Coût au 100km (WLTP)'] = df_pdf['Coût au 100km (WLTP)'].apply(transform_and_extract_numeric)

In [13]:
# On supprime les colonnes qui sont en double
df_pdf.drop(columns=['Capacité Batterie (brute / utile)', 'Puissance/couple moteur(s) ch / kW / Nm', 'Poids à Vide/Cx/Scx', 'Autonomie/consommation constructeur (cycle WLTP)', 'Estimation autonomie / conso. réelle en ville (à 23°C et -10°C)', 'Estimation autonomie / conso. réelle autoroute (à 23°C et -10°)', 'Modes de charge / puissances'], inplace=True)

In [14]:
df_pdf

,Marque / Modèle,Temps 0-100km/h,Vitesse maximale,Coût recharge 100% de la batterie,Coût au 100km (WLTP),Prix du véhicule (TTC - Bonus écologique 6k€ ou 2k€ Inclus),Capacité Batterie brute,Capacité Batterie utile,Puissance/couple moteur(s) ch,Puissance/couple moteur(s) kW,Puissance/couple moteur(s) Nm,Poids à Vide,Poids Cx,Poids Scx,Autonomie,Consommation construsteur (cycle WLTP),Estimation autonomie ville,conso. réelle en ville (à 23°C et -10°C),Estimation autonomie autoroute,conso. réelle autoroute (à 23°C et -10°)
0,Citroën Ami (2020) (FWD) >14 ans (2020),N/D,45 km/h,"0,82€ @ 1,8kW",1.09,6 000 €,"5,8kWh","5,35kWh",12 ch,9 kW,40 Nm,471 kg,?,?,75,"78,6 Wh/km",NaN,NaN,NaN,NaN
1,Lightyear One (2022) (AWD),10 sec.,150 km/h,"8,85€ @ 22kW",1.18,149 000 €,60kWh,60kWh,136 ch,100 kW,1200 Nm,1300 kg,"0,2","0,46",725,83 Wh/km,875-505 km,69-119 Wh/km,565-405 km,106-148 Wh/km
2,Renault Twingo Electric (RWD) (2020),"12,6 sec.",135 km/h,"3,14€ @ 22kW",1.65,15 350 - 20 450 €,23 kWh,"21,3 kWh",82 ch,60 kW,160 Nm,1208kg,"0,33","0,72",190,"112,1 Wh/km",205-135 km,104-158 Wh/km,120-90 km,178-237 Wh/km
3,Tesla Model 3 - Std range+ (NCA) (RWD)(2019),"5,6 sec.",225 km/h,"7,52€@11kW / 12,24€ (SUC)",1.67,43 800 €,54kWh,51kWh,325 ch,239 kW,420 Nm,1700kg,"0,23","0,52",448,"113,8 Wh/km",515-330 km,99-155 Wh/km,330-250 km,155-204 Wh/km
4,Fiat 500e Hatchback 42 kWh (FWD) (2020),9 sec.,150 km/h,"6,2€ @ 11kW",1.93,23 990 - 28900 €,42 kWh,"37,3 kWh",118 ch,87 kW,220 Nm,1465kg,"0,31","0,65",320,"116,5 Wh/km",375-240 km,99-155 Wh/km,225-175 km,175-230 Wh/km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Byton M-Byte 95 kWh (Chine) (AWD) (2022),"5,5 sec.",190 km/h,14€ @ 11kW,3.22,~ 65 000 €,105 kWh,95 kWh,408 ch,300 kW,735 Nm,2600kg,?,?,435,218 Wh/km,565-400 km,168-238 Wh/km,350-280 km,271-339 Wh/km
153,Mercedes EQV 300 Long (8 places) (FWD) (2019),10 sec.,140 km/h,"13,3€ @ 11kW",3.66,72 744 €,100kWh,90kWh,204 ch,150 kW,362 Nm,2635kg,?,?,363,"247,9 Wh/km",480-350 km,188-257 Wh/km,290-235 km,310-383 Wh/km
154,Tesla Cybertruck Single Motor (RWD) (2022),7 sec.,180 km/h,"14,8€ @ 11kW / 24€ (SUC)",3.78,43000 € ?,100 kWh,100 kWh,? ch,? kW,? Nm,2600kg,?,?,390,256 Wh/km,565-405 km,177-247 Wh/km,345-275 km,290-364 Wh/km
155,Tesla Cybertruck Dual Motor (AWD) (2022),"4,5 sec.",193 km/h,"17,7€ @ 11kW / 28,8€ (SUC)",3.85,53000 € ?,120 kWh,120 kWh,? ch,? kW,? Nm,2750kg,?,?,460,261 Wh/km,660-480 km,182-250 Wh/km,405-325 km,293-369 Wh/km


In [15]:
df_pdf.to_csv("comparatif_VE.csv", index=False)